In [13]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors

# --- Constants ---

INPUT_DIR = "/home/aricept094/mydata/ANOVA/radius3"
OUTPUT_DIR = "/home/aricept094/mydata/ANOVA/results"
SECTION_NAMES = [
    "dc_component",  # keep dc_component as it is in csv
    "component_1_amplitude",
    "component_2_amplitude",
    "higher_order_amplitude_sum"
]
SECTION_DISPLAY_NAMES = [  # list for display names
    "dc_component_amplitude",  # display as dc_component_amplitude
    "component_1_amplitude",
    "component_2_amplitude",
    "higher_order_amplitude_sum"
]
BAR_NAMES = [
    "casia_less_than_1",
    "casia1-2",
    "casia2-4",
    "casia_more_than_4"
]
LEGEND_DISPLAY_NAMES = [  # new list for legend display names
    "under 1 diopter",
    "1 - 2 diopter",
    "2 - 4 diopter",
    "4 - 6 diopter"
]
NUMBERS = [4, 8, 12, 16, 20, 24]
# Dictionary to map radius numbers to their mm values
RADIUS_MM_MAP = {
    4: "0.5 mm",
    8: "1 mm",
    12: "1.5 mm",
    16: "2 mm",
    20: "2.5 mm",
    24: "3 mm"
}

LOG_SCALE_MIN = 1e-8
LOG_SCALE_MARGIN = 2  # margin factor for log scale

# Ensure output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

group_colors = {
    "casia_less_than_1": "#E64B35",  # Red
    "casia1-2": "#4DBBD5",           # Blue
    "casia2-4": "#00A087",           # Teal
    "casia_more_than_4": "#3C5488"   # Navy
}

def format_value(value):
    if value >= 1:
        return f"{value:.3f}"
    elif value >= 0.001:
        return f"{value:.3f}"
    else:
        return f"{value:.2e}"

def read_and_process_data(input_dir, numbers, group_names, parameters_of_interest):
    all_data = []
    for number in numbers:
        for group_name in group_names:
            filename = f"analysis_results_radial_{number}_{group_name}.csv"
            filepath = os.path.join(input_dir, filename)
            try:
                df = pd.read_csv(filepath)
                for parameter in parameters_of_interest:
                    if parameter in df.columns:  # check if parameter exists in df columns
                        mean_val = df[parameter].mean()
                        std_val = df[parameter].std()
                        all_data.append({
                            'radius': number,
                            'group': group_name,
                            'parameter': parameter,
                            'mean_value': mean_val,
                            'std_value': std_val
                        })
                    else:
                        print(f"Warning: Parameter '{parameter}' not found in file: {filepath}")
            except FileNotFoundError:
                print(f"Warning: File not found: {filepath}")
    return pd.DataFrame(all_data)

def calculate_y_limits(data_df):
    min_y_value = float('inf')
    max_y_value = float('-inf')

    for _, row in data_df.iterrows():
        mean_val = row['mean_value']
        std_val = row['std_value']

        # Handle non-positive values for log scale: use a tiny value
        if mean_val <= 0:
            mean_val = LOG_SCALE_MIN
        if std_val <= 0:
            std_val = LOG_SCALE_MIN

        min_y_value = min(min_y_value, mean_val - std_val if not np.isnan(std_val) else mean_val)
        max_y_value = max(max_y_value, mean_val + std_val if not np.isnan(std_val) else mean_val)

    # Ensure limits are appropriate for log scale
    if min_y_value == float('inf') or min_y_value <= 0:
        min_y_value = LOG_SCALE_MIN
    if max_y_value == float('-inf') or max_y_value <= 0:
        max_y_value = 1
    if min_y_value == max_y_value:
        y_min = min_y_value / 10
        y_max = max_y_value * 10
    else:
        y_min = min_y_value/LOG_SCALE_MARGIN
        y_max = max_y_value*LOG_SCALE_MARGIN

    return y_min, y_max

def plot_bars(ax, radius_data, bar_width, section_positions, y_max, section_names, bar_names, section_display_names):
    num_bars = len(bar_names)
    for i, section_name in enumerate(section_names):
        section_data = radius_data[radius_data['parameter'] == section_name]
        section_means = []
        section_stds = []

        for bar_name in bar_names:
            bar_row = section_data[section_data['group'] == bar_name]
            if not bar_row.empty:
                mean_val = bar_row['mean_value'].iloc[0]
                std_val = bar_row['std_value'].iloc[0]

                if mean_val <= 0:
                    mean_val = LOG_SCALE_MIN
                if std_val <= 0:
                    std_val = LOG_SCALE_MIN
                section_means.append(mean_val)
                section_stds.append(std_val)
            else:
                section_means.append(LOG_SCALE_MIN)
                section_stds.append(LOG_SCALE_MIN)

        bar_positions_in_section = section_positions[i] + np.arange(num_bars) * bar_width - (num_bars * bar_width / 2) + bar_width / 2
        section_stds_non_neg = [std if not np.isnan(std) else LOG_SCALE_MIN for std in section_stds]

        # Create bars with error bars
        bars = ax.bar(bar_positions_in_section, section_means, width=bar_width,
                     label=None,  # removed section based label
                     color=[group_colors[bar_name] for bar_name in bar_names], alpha=0.8,  # color based on bar_name (group)
                     edgecolor='black', linewidth=0.7,
                     yerr=section_stds_non_neg, capsize=5)

        # Add value labels with improved formatting
        for bar, mean_value in zip(bars, section_means):
            yval = bar.get_height()
            formatted_value = format_value(yval)

            # Position the text and adjust font size based on bar height
            if yval > y_max / 100:  # For taller bars
                text_y_pos = yval / 2
                fontsize = 8
                rotation = 0
            else:  # For shorter bars
                text_y_pos = yval * 1.1
                fontsize = 7
                rotation = 0

            # Add a white background to text for better readability
            ax.text(bar.get_x() + bar.get_width()/2, text_y_pos,
                   formatted_value,
                   ha='center', va='center',
                   fontsize=fontsize, rotation=rotation,
                   bbox=dict(facecolor='white', alpha=0.7, edgecolor='none', pad=1))

def plot_horizontal_lines(ax, line_df, radius_num, mean_values_df, section_positions, bar_width, max_y_value, new_y_max, section_names, bar_names):
    lines_to_draw = line_df[line_df['radial_number'] == radius_num]
    num_sections = len(section_names)
    num_bars = len(bar_names)
    line_spacing = new_y_max / (len(lines_to_draw) + 2) if lines_to_draw.shape[0] > 0 else new_y_max / 10  # Adjust spacing based on the number of lines
    current_line_y_position = new_y_max * 0.9  # start just below the top margin
    drawn_lines_y_positions = {}  # To track y positions for each line group

    if current_line_y_position <= 0:
        current_line_y_position = LOG_SCALE_MIN * 10  # Ensure it's above zero even in log scale

    line_counter = 0  # Counter for vertical line positioning

    for _, row in lines_to_draw.iterrows():
        group1_name = row['group1']
        group2_name = row['group2']
        param_name = row['parameter']

        if param_name not in section_names:
            continue

        try:
            group1_index = bar_names.index(group1_name)
            group2_index = bar_names.index(group2_name)
            param_index = section_names.index(param_name)
        except ValueError:
            print(f"Warning: Group or parameter name not found: {group1_name}, {group2_name}, or {param_name}")
            continue

        # Calculate x positions for bar centers
        x_start_bar_center = section_positions[param_index] + (group1_index * bar_width) - (num_bars * bar_width / 2) + bar_width / 2
        x_end_bar_center = section_positions[param_index] + (group2_index * bar_width) - (num_bars * bar_width / 2) + bar_width / 2

        # Use line counter to adjust y position for vertical spacing
        current_line_y_position_adjusted = current_line_y_position - (line_counter * line_spacing)
        if current_line_y_position_adjusted <= 0:  # very important to keep lines in positive range for log scale
            current_line_y_position_adjusted = current_line_y_position - (line_counter * line_spacing) + LOG_SCALE_MIN * 10  # add a small offset

        line_width = 1.5

        ax.hlines(y=current_line_y_position_adjusted, xmin=x_start_bar_center, xmax=x_end_bar_center,
                 color='black', linestyle='-', linewidth=line_width)
        line_counter += 1  # Increment counter for next line

def create_bar_chart_improved(radius_num, data_df, line_df, output_path, section_names, bar_names, section_display_names, legend_display_names):
    radius_data = data_df[data_df['radius'] == radius_num]
    if radius_data.empty:
        print(f"No data available for radius {radius_num}. Skipping plot.")
        return

    # Set style for better readability
    plt.style.use('seaborn-v0_8-whitegrid')

    fig, ax1 = plt.subplots(figsize=(14, 7))

    num_sections = len(section_names)
    num_bars = len(bar_names)
    bar_width = 0.8 / num_bars
    section_positions = np.arange(num_sections)

    y_min, y_max = calculate_y_limits(radius_data)
    plot_bars(ax1, radius_data, bar_width, section_positions, y_max, section_names, bar_names, section_display_names)

    # Add top margin for significance lines
    current_y_min, current_y_max = ax1.get_ylim()
    top_margin_factor = 1.5  # Adjusted for linear space, reduce from 3.7
    new_y_max = current_y_max * top_margin_factor
    ax1.set_ylim(current_y_min, new_y_max)

    # Improve axis appearance
    ax1.set_yscale('log')
    ax1.set_ylim(y_min, new_y_max)  # Use new_y_max for ylim
    ax1.set_xticks(section_positions)

    # Use display names for x tick labels
    ax1.set_xticklabels([name.replace('_', ' ').title() for name in section_display_names],
                        fontsize=10, rotation=0, ha='center')

    # Improve labels and title
    ax1.set_ylabel('Mean Value (Log Scale)', fontsize=12, fontweight='bold')
    ax1.set_xlabel('Parameter Component', fontsize=12, fontweight='bold')
    ax1.set_title(f'Mean Parameter Values by Component for Radius {RADIUS_MM_MAP[radius_num]}',  # Changed title to show mm value
                  fontsize=14, fontweight='bold', pad=20)

    # Add grid with custom style
    ax1.grid(True, which='both', linestyle='--', alpha=0.6)

    # Add horizontal lines for statistical significance
    plot_horizontal_lines(ax1, line_df, radius_num, data_df, section_positions, bar_width, y_max, new_y_max, section_names, bar_names)

    # Improve legend
    legend_elements = [plt.Rectangle((0, 0), 1, 1, color=group_colors[group]) for group in bar_names]
    legend_labels = legend_display_names

    legend = ax1.legend(legend_elements, legend_labels,
                       title='Groups', loc='upper right',
                       bbox_to_anchor=(1.0, 0.99),
                       frameon=True,
                       fontsize=7.5, title_fontsize=9)
    legend.get_frame().set_alpha(0.9)
    legend.get_frame().set_edgecolor('gray')

    # Adjust layout and save
    plt.tight_layout(rect=[0, 0, 1, 0.85])
    plt.savefig(output_path, dpi=950, bbox_inches='tight')
    plt.close()
    print(f"Improved bar chart saved to: {output_path}")

# --- Main Script ---
if __name__ == "__main__":
    # --- Data Reading and Mean and STD Calculation ---
    mean_values_df = read_and_process_data(INPUT_DIR, NUMBERS, BAR_NAMES, SECTION_NAMES)

    # i will fill these my self, do not remove
    line_data = pd.DataFrame({
        'group1': ['casia1-2', 'casia2-4', 'casia_less_than_1', 'casia1-2', 'casia_less_than_1', 'casia1-2',
                'casia_less_than_1', 'casia1-2', 'casia1-2', 'casia_less_than_1', 'casia_less_than_1',
                'casia1-2', 'casia2-4', 'casia_less_than_1', 'casia1-2', 'casia_less_than_1', 'casia2-4',
                'casia_less_than_1', 'casia1-2', 'casia1-2', 'casia2-4', 'casia_less_than_1', 'casia1-2',
                'casia_less_than_1', 'casia_less_than_1', 'casia1-2', 'casia2-4', 'casia1-2', 'casia1-2',
                'casia2-4', 'casia_less_than_1', 'casia1-2', 'casia_less_than_1', 'casia1-2', 'casia1-2',
                'casia2-4', 'casia_less_than_1', 'casia1-2', 'casia2-4', 'casia2-4', 'casia_less_than_1',
                'casia1-2', 'casia1-2', 'casia2-4', 'casia_less_than_1', 'casia1-2', 'casia1-2', 'casia2-4',
                'casia_less_than_1'],
        'group2': ['casia_more_than_4', 'casia_more_than_4', 'casia_more_than_4', 'casia_more_than_4',
                'casia_more_than_4', 'casia_more_than_4', 'casia_more_than_4', 'casia2-4', 'casia_more_than_4',
                'casia_more_than_4', 'casia_more_than_4', 'casia_more_than_4', 'casia_more_than_4',
                'casia_more_than_4', 'casia_more_than_4', 'casia_more_than_4', 'casia_less_than_1',
                'casia_more_than_4', 'casia_less_than_1', 'casia_more_than_4', 'casia_more_than_4',
                'casia_more_than_4', 'casia_more_than_4', 'casia_more_than_4', 'casia_more_than_4',
                'casia_more_than_4', 'casia_more_than_4', 'casia2-4', 'casia_more_than_4', 'casia_less_than_1',
                'casia_more_than_4', 'casia_more_than_4', 'casia_more_than_4', 'casia2-4', 'casia_more_than_4',
                'casia_less_than_1', 'casia_more_than_4', 'casia_more_than_4', 'casia_less_than_1',
                'casia_more_than_4', 'casia_more_than_4', 'casia2-4', 'casia_more_than_4', 'casia_less_than_1',
                'casia_more_than_4', 'casia2-4', 'casia_more_than_4', 'casia_less_than_1', 'casia_more_than_4'],
        'radial_number': [4, 4, 4, 4, 4, 4, 4, 8, 8, 8, 8, 8, 8, 8, 12, 12, 12, 12, 12, 12, 12, 12, 16, 16, 16,
                        16, 16, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24,
                        24, 24],
        'parameter': ['component_1_amplitude', 'component_1_amplitude', 'component_1_amplitude', 'component_2_amplitude',
                    'component_2_amplitude', 'higher_order_amplitude_sum', 'higher_order_amplitude_sum',
                    'component_1_amplitude', 'component_1_amplitude', 'component_1_amplitude', 'component_2_amplitude',
                    'higher_order_amplitude_sum', 'higher_order_amplitude_sum', 'higher_order_amplitude_sum',
                    'component_1_amplitude', 'component_1_amplitude', 'component_2_amplitude', 'component_2_amplitude',
                    'higher_order_amplitude_sum', 'higher_order_amplitude_sum', 'higher_order_amplitude_sum',
                    'higher_order_amplitude_sum', 'component_1_amplitude', 'component_1_amplitude',
                    'component_2_amplitude', 'higher_order_amplitude_sum', 'higher_order_amplitude_sum',
                    'component_1_amplitude', 'component_1_amplitude', 'component_1_amplitude', 'component_1_amplitude',
                    'component_2_amplitude', 'component_2_amplitude', 'higher_order_amplitude_sum',
                    'higher_order_amplitude_sum', 'higher_order_amplitude_sum', 'higher_order_amplitude_sum',
                    'component_1_amplitude', 'component_1_amplitude', 'component_1_amplitude', 'component_1_amplitude',
                    'component_2_amplitude', 'component_2_amplitude', 'component_2_amplitude', 'component_2_amplitude',
                    'higher_order_amplitude_sum', 'higher_order_amplitude_sum', 'higher_order_amplitude_sum',
                    'higher_order_amplitude_sum']
    })

    # --- Generate Improved Plots ---
    for number in NUMBERS:
        plot_output_path = os.path.join(OUTPUT_DIR, f"improved_bar_chart_radius_{number}.png")
        lines_for_radius = line_data[line_data['radial_number'] == number]
        create_bar_chart_improved(number, mean_values_df, lines_for_radius, plot_output_path, 
                                SECTION_NAMES, BAR_NAMES, SECTION_DISPLAY_NAMES, LEGEND_DISPLAY_NAMES)

    print("All plots generated successfully.")

Improved bar chart saved to: /home/aricept094/mydata/ANOVA/results/improved_bar_chart_radius_4.png
Improved bar chart saved to: /home/aricept094/mydata/ANOVA/results/improved_bar_chart_radius_8.png
Improved bar chart saved to: /home/aricept094/mydata/ANOVA/results/improved_bar_chart_radius_12.png
Improved bar chart saved to: /home/aricept094/mydata/ANOVA/results/improved_bar_chart_radius_16.png
Improved bar chart saved to: /home/aricept094/mydata/ANOVA/results/improved_bar_chart_radius_20.png
Improved bar chart saved to: /home/aricept094/mydata/ANOVA/results/improved_bar_chart_radius_24.png
All plots generated successfully.


In [ ]:
    line_data = pd.DataFrame({
        'group1': ['casia1-2', 'casia2-4', 'casia_less_than_1', 'casia1-2', 'casia_less_than_1', 'casia1-2',
                'casia_less_than_1', 'casia1-2', 'casia1-2', 'casia_less_than_1', 'casia_less_than_1',
                'casia1-2', 'casia2-4', 'casia_less_than_1', 'casia1-2', 'casia_less_than_1', 'casia2-4',
                'casia_less_than_1', 'casia1-2', 'casia1-2', 'casia2-4', 'casia_less_than_1', 'casia1-2',
                'casia_less_than_1', 'casia_less_than_1', 'casia1-2', 'casia2-4', 'casia1-2', 'casia1-2',
                'casia2-4', 'casia_less_than_1', 'casia1-2', 'casia_less_than_1', 'casia1-2', 'casia1-2',
                'casia2-4', 'casia_less_than_1', 'casia1-2', 'casia2-4', 'casia2-4', 'casia_less_than_1',
                'casia1-2', 'casia1-2', 'casia2-4', 'casia_less_than_1', 'casia1-2', 'casia1-2', 'casia2-4',
                'casia_less_than_1'],
        'group2': ['casia_more_than_4', 'casia_more_than_4', 'casia_more_than_4', 'casia_more_than_4',
                'casia_more_than_4', 'casia_more_than_4', 'casia_more_than_4', 'casia2-4', 'casia_more_than_4',
                'casia_more_than_4', 'casia_more_than_4', 'casia_more_than_4', 'casia_more_than_4',
                'casia_more_than_4', 'casia_more_than_4', 'casia_more_than_4', 'casia_less_than_1',
                'casia_more_than_4', 'casia_less_than_1', 'casia_more_than_4', 'casia_more_than_4',
                'casia_more_than_4', 'casia_more_than_4', 'casia_more_than_4', 'casia_more_than_4',
                'casia_more_than_4', 'casia_more_than_4', 'casia2-4', 'casia_more_than_4', 'casia_less_than_1',
                'casia_more_than_4', 'casia_more_than_4', 'casia_more_than_4', 'casia2-4', 'casia_more_than_4',
                'casia_less_than_1', 'casia_more_than_4', 'casia_more_than_4', 'casia_less_than_1',
                'casia_more_than_4', 'casia_more_than_4', 'casia2-4', 'casia_more_than_4', 'casia_less_than_1',
                'casia_more_than_4', 'casia2-4', 'casia_more_than_4', 'casia_less_than_1', 'casia_more_than_4'],
        'radial_number': [4, 4, 4, 4, 4, 4, 4, 8, 8, 8, 8, 8, 8, 8, 12, 12, 12, 12, 12, 12, 12, 12, 16, 16, 16,
                        16, 16, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24,
                        24, 24],
        'parameter': ['component_1_amplitude', 'component_1_amplitude', 'component_1_amplitude', 'component_2_amplitude',
                    'component_2_amplitude', 'higher_order_amplitude_sum', 'higher_order_amplitude_sum',
                    'component_1_amplitude', 'component_1_amplitude', 'component_1_amplitude', 'component_2_amplitude',
                    'higher_order_amplitude_sum', 'higher_order_amplitude_sum', 'higher_order_amplitude_sum',
                    'component_1_amplitude', 'component_1_amplitude', 'component_2_amplitude', 'component_2_amplitude',
                    'higher_order_amplitude_sum', 'higher_order_amplitude_sum', 'higher_order_amplitude_sum',
                    'higher_order_amplitude_sum', 'component_1_amplitude', 'component_1_amplitude',
                    'component_2_amplitude', 'higher_order_amplitude_sum', 'higher_order_amplitude_sum',
                    'component_1_amplitude', 'component_1_amplitude', 'component_1_amplitude', 'component_1_amplitude',
                    'component_2_amplitude', 'component_2_amplitude', 'higher_order_amplitude_sum',
                    'higher_order_amplitude_sum', 'higher_order_amplitude_sum', 'higher_order_amplitude_sum',
                    'component_1_amplitude', 'component_1_amplitude', 'component_1_amplitude', 'component_1_amplitude',
                    'component_2_amplitude', 'component_2_amplitude', 'component_2_amplitude', 'component_2_amplitude',
                    'higher_order_amplitude_sum', 'higher_order_amplitude_sum', 'higher_order_amplitude_sum',
                    'higher_order_amplitude_sum']
    })
    
    
        line_data = pd.DataFrame({
        'group1': [],
        'group2': [],
        'p_adj': [],
        'radial_number': [],
        'parameter': []
    })

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors

# --- Constants ---

INPUT_DIR = "/home/aricept094/mydata/ANOVA/radius"
OUTPUT_DIR = "/home/aricept094/mydata/ANOVA/results"
PARAMETERS_OF_INTEREST = [
    "dc_component",
    "component_1_amplitude",
    "component_2_amplitude",
    "higher_order_amplitude_sum"
]
GROUP_NAMES = [
    "casia_less_than_1",
    "casia1-2",
    "casia2-4",
    "casia_more_than_4"
]
NUMBERS = [4, 8, 12, 16, 20, 24]

LOG_SCALE_MIN = 1e-8
LOG_SCALE_MARGIN = 2  # margin factor for log scale

# Ensure output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

def format_value(value):
    """
    Format values to be more readable:
    - For values >= 1: show up to 3 decimal places
    - For values < 1: use scientific notation with 2 significant figures
    - For very small values (< 0.001): use scientific notation
    """
    if value >= 1:
        return f"{value:.3f}"
    elif value >= 0.001:
        return f"{value:.3f}"
    else:
        return f"{value:.2e}"

def read_and_process_data(input_dir, numbers, group_names, parameters_of_interest):
    """Reads CSV files, calculates mean and std, and returns a DataFrame."""
    all_data = []
    for number in numbers:
        for group_name in group_names:
            filename = f"analysis_results_radial_{number}_{group_name}.csv"
            filepath = os.path.join(input_dir, filename)
            try:
                df = pd.read_csv(filepath)
                for parameter in parameters_of_interest:
                    mean_val = df[parameter].mean()
                    std_val = df[parameter].std()
                    all_data.append({
                        'radius': number,
                        'group': group_name,
                        'parameter': parameter,
                        'mean_value': mean_val,
                        'std_value': std_val
                    })
            except FileNotFoundError:
                print(f"Warning: File not found: {filepath}")
    return pd.DataFrame(all_data)

def calculate_y_limits(data_df):
    """Calculates the y-axis limits based on mean and standard deviation values."""
    min_y_value = float('inf')
    max_y_value = float('-inf')

    for _, row in data_df.iterrows():
        mean_val = row['mean_value']
        std_val = row['std_value']

        # Handle non-positive values for log scale: use a tiny value
        if mean_val <= 0:
            mean_val = LOG_SCALE_MIN
        if std_val <= 0:
            std_val = LOG_SCALE_MIN

        min_y_value = min(min_y_value, mean_val - std_val if not np.isnan(std_val) else mean_val)
        max_y_value = max(max_y_value, mean_val + std_val if not np.isnan(std_val) else mean_val)

    # Ensure limits are appropriate for log scale
    if min_y_value == float('inf') or min_y_value <= 0:
        min_y_value = LOG_SCALE_MIN
    if max_y_value == float('-inf') or max_y_value <= 0:
        max_y_value = 1
    if min_y_value == max_y_value:
        y_min = min_y_value / 10
        y_max = max_y_value * 10
    else:
        y_min = min_y_value/LOG_SCALE_MARGIN
        y_max = max_y_value*LOG_SCALE_MARGIN

    return y_min, y_max


def plot_bars(ax, radius_data, bar_width, group_positions, parameter_colors, y_max):
    """Plots the bars and value labels for each parameter with improved formatting."""
    num_parameters = len(PARAMETERS_OF_INTEREST)
    for i, parameter in enumerate(PARAMETERS_OF_INTEREST):
        parameter_data = radius_data[radius_data['parameter'] == parameter]
        parameter_means = []
        parameter_stds = []

        for group in GROUP_NAMES:
            group_row = parameter_data[parameter_data['group'] == group]
            if not group_row.empty:
                mean_val = group_row['mean_value'].iloc[0]
                std_val = group_row['std_value'].iloc[0]

                if mean_val <= 0:
                    mean_val = LOG_SCALE_MIN
                if std_val <= 0:
                    std_val = LOG_SCALE_MIN
                parameter_means.append(mean_val)
                parameter_stds.append(std_val)
            else:
                parameter_means.append(LOG_SCALE_MIN)
                parameter_stds.append(LOG_SCALE_MIN)

        bar_positions = group_positions + (i * bar_width) - (num_parameters * bar_width / 2) + bar_width / 2
        parameter_stds_non_neg = [std if not np.isnan(std) else LOG_SCALE_MIN for std in parameter_stds]

        # Create bars with error bars
        bars = ax.bar(bar_positions, parameter_means, width=bar_width,
                     label=parameter.replace('_', ' ').title(),
                     color=parameter_colors[i], alpha=0.8,
                     edgecolor='black', linewidth=0.7,
                     yerr=parameter_stds_non_neg, capsize=5)

        # Add value labels with improved formatting
        for bar, mean_value in zip(bars, parameter_means):
            yval = bar.get_height()
            formatted_value = format_value(yval)

            # Position the text and adjust font size based on bar height
            if yval > y_max / 100:  # For taller bars
                text_y_pos = yval / 2
                fontsize = 8
                rotation = 0
            else:  # For shorter bars
                text_y_pos = yval * 1.1
                fontsize = 7
                rotation = 0

            # Add a white background to text for better readability
            ax.text(bar.get_x() + bar.get_width()/2, text_y_pos,
                   formatted_value,
                   ha='center', va='center',
                   fontsize=fontsize, rotation=rotation,
                   bbox=dict(facecolor='white', alpha=0.7, edgecolor='none', pad=1))


def plot_horizontal_lines(ax, line_df, radius_num, mean_values_df, group_positions, bar_width, max_y_value, new_y_max):
    """Plots the horizontal lines based on the line_df with y-axis spacing."""
    lines_to_draw = line_df[line_df['radial_number'] == radius_num]
    num_parameters = len(PARAMETERS_OF_INTEREST)
    line_spacing = new_y_max / 10 # use new_y_max for spacing
    current_line_y_position = new_y_max * 0.9 # start just below the top margin


    if current_line_y_position <= 0:
        current_line_y_position = LOG_SCALE_MIN

    drawn_lines_y_positions = {}
    for _, row in lines_to_draw.iterrows():
        group1_name = row['group1']
        group2_name = row['group2']
        param_name = row['parameter']


        if param_name not in PARAMETERS_OF_INTEREST:
            continue

        try:
            group1_index = GROUP_NAMES.index(group1_name)
            group2_index = GROUP_NAMES.index(group2_name)
            param_index = PARAMETERS_OF_INTEREST.index(param_name)
        except ValueError:
            print(f"Warning: Group or parameter name not found: {group1_name}, {group2_name}, or {param_name}")
            continue

        # Calculate x positions for bar centers
        x_start_bar_center = group_positions[group1_index] + (param_index * bar_width) - (num_parameters * bar_width / 2) + bar_width / 2
        x_end_bar_center = group_positions[group2_index] + (param_index * bar_width) - (num_parameters * bar_width / 2) + bar_width / 2


         # Check if a line with the same x positions has been drawn
        line_key = tuple(sorted([x_start_bar_center, x_end_bar_center]))

        if line_key in drawn_lines_y_positions:
            # Increment y position if a line with same x positions is drawn, decrement for downwards spacing in y axis
            current_line_y_position = drawn_lines_y_positions[line_key] - line_spacing

        else:
             drawn_lines_y_positions[line_key] = current_line_y_position

        line_width = 1.5


        ax.hlines(y=current_line_y_position, xmin=x_start_bar_center, xmax=x_end_bar_center,
                 color='black', linestyle='-', linewidth=line_width)



def create_bar_chart_improved(radius_num, data_df, line_df, output_path):
    """Creates an improved bar chart with better formatted values."""
    radius_data = data_df[data_df['radius'] == radius_num]
    if radius_data.empty:
        print(f"No data available for radius {radius_num}. Skipping plot.")
        return

    # Set style for better readability
    plt.style.use('seaborn-v0_8-whitegrid')

    fig, ax1 = plt.subplots(figsize=(14, 7))

    num_groups = len(GROUP_NAMES)
    num_parameters = len(PARAMETERS_OF_INTEREST)
    bar_width = 0.8 / num_parameters
    group_positions = np.arange(num_groups)

    # Use colorblind-friendly colors
    parameter_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']

    y_min, y_max = calculate_y_limits(radius_data)
    plot_bars(ax1, radius_data, bar_width, group_positions, parameter_colors, y_max)

    # Add top margin for significance lines
    current_y_min, current_y_max = ax1.get_ylim()
    top_margin_factor = 2.5  # Adjusted for linear space, reduce from 3.7
    new_y_max = current_y_max * top_margin_factor
    ax1.set_ylim(current_y_min, new_y_max)


    # Improve axis appearance
    ax1.set_yscale('log')
    ax1.set_ylim(y_min, new_y_max) # Use new_y_max for ylim
    ax1.set_xticks(group_positions)
    ax1.set_xticklabels([name.replace('_', ' ').title() for name in GROUP_NAMES],
                        fontsize=10, rotation=45, ha='right')

    # Improve labels and title
    ax1.set_ylabel('Mean Value (Log Scale)', fontsize=12, fontweight='bold')
    ax1.set_xlabel('Group', fontsize=12, fontweight='bold')
    ax1.set_title(f'Mean Parameter Values by Group for Radius {radius_num}',
                  fontsize=14, fontweight='bold', pad=20)

    # Add grid with custom style
    ax1.grid(True, which='both', linestyle='--', alpha=0.6)

    # Add horizontal lines for statistical significance, pass new_y_max
    plot_horizontal_lines(ax1, line_df, radius_num, data_df, group_positions, bar_width, y_max, new_y_max)

    # Improve legend
    lines_1, labels_1 = ax1.get_legend_handles_labels()
    unique_labels = dict(zip(labels_1, lines_1))
    legend = ax1.legend(unique_labels.values(), unique_labels.keys(),
                       title='Parameters', loc='upper right',
                       bbox_to_anchor=(1.0, 1.0), frameon=True,
                       fontsize=9, title_fontsize=10)
    legend.get_frame().set_alpha(0.9)
    legend.get_frame().set_edgecolor('gray')

    # Adjust layout and save
    plt.tight_layout()
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Improved bar chart saved to: {output_path}")


# --- Main Script ---

if __name__ == "__main__":
    # --- Data Reading and Mean and STD Calculation ---
    mean_values_df = read_and_process_data(INPUT_DIR, NUMBERS, GROUP_NAMES, PARAMETERS_OF_INTEREST)

    # --- Line Data DataFrame ---
    line_data = pd.DataFrame({
        'group1': ['casia1-2', 'casia_less_than_1', 'casia1-2', 'casia1-2', 'casia_less_than_1', 'casia2-4'],
        'group2': ['casia_more_than_4', 'casia_more_than_4', 'casia2-4', 'casia_more_than_4', 'casia_more_than_4', 'casia_more_than_4'],
        'radial_number': [4, 4, 8, 8, 8, 8],
        'parameter': ['component_1_amplitude', 'component_1_amplitude', 'component_1_amplitude', 'component_1_amplitude', 'component_1_amplitude', 'component_1_amplitude']
    })

    # --- Generate Improved Plots ---
    for number in NUMBERS:
        plot_output_path = os.path.join(OUTPUT_DIR, f"improved_bar_chart_radius_{number}.png")
        lines_for_radius = line_data[line_data['radial_number'] == number]
        create_bar_chart_improved(number, mean_values_df, lines_for_radius, plot_output_path)

    print("All plots generated successfully.")